## Toy data set

Let's train a network to classify ice hockey match scores to match results. We'll have the teams' goal amounts as the input and one-hot encoding of HOME_WIN, DRAW and AWAY_WIN as output. For example, for a match that ends 6–4, we would have [6, 4] as the input and [1, 0, 0] as output, because it's a win for the home team.

In [ ]:
training_data = [
    ([5, 4], [1, 0, 0]),
    ([1, 4], [0, 0, 1]),
    ([2, 4], [0, 0, 1]),
    ([0, 0], [0, 1, 0]),
    ([2, 2], [0, 1, 0]),
    ([7, 1], [1, 0, 0]),
    ([1, 0], [1, 0, 0]),
    ([1, 2], [0, 0, 1])
]

test_data = [
    ([8, 2], [1, 0, 0]),
    ([7, 7], [0, 1, 0]),
    ([1, 3], [0, 0, 1]),
    ([3, 3], [0, 1, 0])
]

training_inputs, training_labels = zip(*training_data)
test_inputs, test_labels = zip(*test_data)

Let's plot the data.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

p, (training_plot, test_plot) = plt.subplots(
        1, 2, figsize=(10.5,5), 
        subplot_kw={'xlim': [-0.2, 8.2], 'ylim': [-0.2, 8.2]})

x, y = zip(*training_inputs)
training_plot.scatter(x, y, c=training_labels, s=100)
training_plot.set_title("training")

x, y = zip(*test_inputs)
test_plot.scatter(x, y, c=test_labels, s=100)
test_plot.set_title("test")

## Softmax classifier

We are going to use a softmax classifier to solve this task. Here are two ways to visualize what we will build:

![](images/softmax for toy problem.png)

Now let's use TensorFlow to build this model.

In [ ]:
### Softmax implementation comes here